In [ ]:
import os
import sys
sys.path.append(os.path.abspath('..'))
from config import *

!cp ../data/acorns.clean.csv .
!cp ../data/cinsueta_exp.csv .

# plotly

Plotly はインタラクティブなグラフを簡単に作成できるライブラリです。Python だけでなく、統計解析に強い R など他の言語からも利用できるため、多くのユーザーに支持されています。数ある可視化ライブラリの中でも、Plotly は一度は触れておく価値のあるツールです。本節では、Plotly を使って基本的なグラフを作成する方法を紹介します。

## plotly ライブラリー

Plotly には主に 2 種類の使用方法があります。一つは、`plotly.graph_objects` モジュールを利用した方法です。高度なカスタマイズが可能で、柔軟性が高い反面、コード量が多くなりがちです。もう一つが `plotly.express` モジュールを利用する方法です。少ないコードで手軽にグラフを描けるシンプルなインターフェースです。Seaborn に似ており、データフレームを与えて、列名で x 軸および y 軸を指定して描画します。ここでは、簡潔に使える `plotly.express` を中心に紹介します。

このモジュールを利用するには次のようにインポートします。

In [ ]:
import plotly.express as px

続けて、可視化用にどんぐりのデータセットとタネツケバナの遺伝子発現量データセットを読み込みます。どんぐりのデータセット（acorns.clean.csv）には、さまざまな種類のどんぐり（樹種）に関する情報が記録されています。各サンプルについて、樹種（tree）、重さ（weight）、高さ（height）、直径（diameter）のデータが記録されています。

In [ ]:
# !wget https://py.biopapyrus.jp/data/acorns.clean.csv
acorn_data = pd.read_csv('acorns.clean.csv')
acorn_data.head()

タネツケバナの遺伝子発現量データ（cinsueta_exp.csv）は、_Cardamine insueta_ の葉を水面に浮かべたあと、経過時間（0〜96時間）ごとの遺伝子発現量の変化を測定したデータです。

In [ ]:
# !wget https://py.biopapyrus.jp/data/cinsueta_exp.csv
exp_data = pd.read_csv('cinsueta_exp.csv', index_col=0)
exp_data = exp_data.reset_index()
exp_data.head()

## 散布図

Plotly Express では、`scatter` 関数を使って散布図を描きます。たとえば、どんぐりの高さを x 軸、重さを y 軸としてプロットしたい場合は、次のように書きます。

In [ ]:
fig = px.scatter(acorn_data, x='height', y='weight')
fig.show()

また、`color` オプションにデータフレームの列名を指定することで、点の色をその列のカテゴリや値に応じて自動で変更できます。`size` オプションを利用すると、各点のサイズも調整可能です。次は、点の色をどんぐりの樹種（tree 列）で変化させ、点のサイズをどんぐりの直径（diameter）に応じて変化させている例です。

In [ ]:
fig = px.scatter(acorn_data, x='height', y='weight', color='tree', size='diameter')
fig.show()

## 線グラフ

タネツケバナの遺伝子発現量データを用いて、特定の遺伝子における発現量の時間変化を折れ線グラフで可視化してみましょう。`exp_data` データフレームは、行に遺伝子、列に時間ごとの発現量が格納された形式になっています。しかし、このままでは x 軸を時間、y 軸を発現量として、列名で指定できません。そこで、以下のように `pd.melt` 関数を用いてデータフレームの形を変えます。

In [ ]:
exp_data_long = pd.melt(exp_data.iloc[0:5, ], id_vars='gene', var_name='timepoint', value_name='exp')

続けて、発現量を対数化し[^logexp]、時間ラベルを整数に変換します。

[^logexp]: 遺伝子発現量のデータは、値の幅が広いため、可視化の前に対数変換を行うのが一般的です。これにより、極端に大きい値による変動幅の偏りを抑え、全体の変化をより見やすくすることができます。この例でも、 NumPy の `np.log10` 関数を使って、遺伝子発現量の値を対数化してからグラフを作成しています。

In [ ]:
exp_data_long['log10exp'] = np.log10(exp_data_long['exp'] + 1)
exp_data_long['timepoint'] = exp_data_long['timepoint'].str.replace('h', '').astype(int)
exp_data_long

データの整形が完了したら、`plotly.express` の `line` 関数を使って、遺伝子ごとの発現変化の時系列変化を折れ線グラフで可視化します。

In [ ]:
fig = px.line(exp_data_long, x='timepoint', y='log10exp', color='gene')
fig.show()

## 棒グラフ

どんぐりのデータを使って、樹種ごとの重さの平均を棒グラフにして視覚化する方法を紹介します。まず、`acorn_data` を樹種（tree）ごとに集計し、それぞれの測定項目の平均値を求めます。

In [ ]:
acorn_weight = acorn_data.groupby('tree').agg(np.mean).reset_index()
acorn_weight

重さ（weight）の平均値を縦棒グラフで可視化します。x 軸に樹種、y 軸に平均重さを指定します。

In [ ]:
fig = px.bar(acorn_weight, x='tree', y='weight')
fig.show()

重さ以外の測定項目（height や diameter など）も含めて比較したい場合は、測定項目が列名として指定できるようにデータフレームの形を変更します。

In [ ]:
acorn_weight_long = pd.melt(acorn_weight, id_vars='tree', var_name='item', value_name='value')
acorn_weight_long.head()

続けて、整形後のデータを使って、item 列の値（重さ、直径、高さなど）ごとに色分けした横並びの棒グラフを描きます。`barmode='group'` を指定することで、グループごとに棒が並びます。

In [ ]:
fig = px.bar(acorn_weight_long, x='tree', y='value', color='item', barmode='group')
fig.show()

なお、`barmode` を省略した場合は、積み上げ棒グラフが描かれます。

In [ ]:
fig = px.bar(acorn_weight_long, x='tree', y='value', color='item')
fig.show()

## ヒストグラム

`histogram` 関数を使うと、数値データの分布をヒストグラムで可視化できます。たとえば、どんぐりの重さ（weight）の分布を表示するには、次のように書きます。

In [ ]:
fig = px.histogram(acorn_data, x='weight')
fig.show()

`color` オプションを使うと、どんぐりの樹種（tree）ごとに色分けしたヒストグラムを描くことができます。複数個のヒストグラムがある場合、`barmode='overlay'` を使うことで、重ねて表示されるようになります。また、`nbins` でビン（棒）の数を指定することができます。

In [ ]:
fig = px.histogram(acorn_data, x='weight', color='tree', nbins=20, barmode='overlay')
fig.show()

`histogram` 関数の `marginal` オプションを使えば、ヒストグラムの上部にボックスプロットを追加することができます。マウスをボックスプロットに載せると、各種の要約統計量を確認することができます。

In [ ]:
fig = px.histogram(acorn_data, x='weight', color='tree', nbins=20, barmode='overlay', marginal='box')
fig.show()

In [ ]:
!rm acorns.clean.csv
!rm cinsueta_exp.csv